In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import mannwhitneyu
from sklearn.metrics import roc_auc_score

In [2]:

df_new = pd.read_excel('dataset_780.xlsx',sheet_name=0)


print(df_new['Appendix_on_US'].value_counts())

print(df_new.shape)

print(df_new['CRP'].dtype)

Appendix_on_US
yes    504
no     273
Name: count, dtype: int64
(782, 58)
float64


In [3]:
# for col in df_new.columns:
#     x=df_new[col].isnull().sum()
#     print(f'Nan in {col}: {x}')

drop_col1=['Diagnosis_Presumptive','Segmented_Neutrophils',
          'Appendix_Wall_Layers','Target_Sign','Appendicolith','Perfusion','Perforation',
          'Surrounding_Tissue_Reaction','Appendicular_Abscess','Conglomerate_of_Bowel_Loops','Coprostasis','Meteorism','Gynecological_Findings',
          'Abscess_Location','Pathological_Lymph_Nodes','Appendix_on_US','Management',
          'US_Performed','US_Number','Length_of_Stay','Lymph_Nodes_Location','Bowel_Wall_Thickening','Ileus','Enteritis']

df_new=df_new.drop(columns=drop_col1)


all_col=['Age', 'BMI', 'Sex', 'Height', 'Weight',
         
       'Migratory_Pain', 'Coughing_Pain', 'Nausea', 'Loss_of_Appetite','Dysuria', 'Stool','Lower_Right_Abd_Pain',
       'Ipsilateral_Rebound_Tenderness','Contralateral_Rebound_Tenderness','Psoas_Sign','Peritonitis', 
        'Body_Temperature', 'WBC_Count','Neutrophilia',
       'Neutrophil_Percentage', 'RBC_Count','Thrombocyte_Count', 'Hemoglobin', 'RDW',
        'Ketones_in_Urine', 'RBC_in_Urine', 'WBC_in_Urine',
       'CRP',  
       'Appendix_Diameter','Free_Fluids','Alvarado_Score','Paedriatic_Appendicitis_Score','Diagnosis', 'Severity']
df_new=df_new[all_col]

feat_cat=['Sex','Migratory_Pain','Coughing_Pain','Nausea','Loss_of_Appetite','Dysuria','Stool','Neutrophilia',
         'Lower_Right_Abd_Pain','Ipsilateral_Rebound_Tenderness','Contralateral_Rebound_Tenderness',
         'Free_Fluids','Psoas_Sign','Peritonitis','Ketones_in_Urine', 'RBC_in_Urine', 'WBC_in_Urine','Diagnosis', 'Severity']

feat_cont=['Age', 'BMI','Height', 'Weight','Body_Temperature', 'WBC_Count','Neutrophil_Percentage', 
           'RBC_Count','Thrombocyte_Count', 'Hemoglobin', 'RDW','CRP','Appendix_Diameter','Alvarado_Score','Paedriatic_Appendicitis_Score','Diagnosis', 'Severity']


# print(len(all_col))
# print(len(feat_cat))
# print(len(feat_cont))



# df_new.to_excel('draft1.xlsx',index=False)
# print(df_new.columns)

# for col in df_new.columns:
#     x=df_new[col].isna().sum()
#     print(f'Nan in {col} : {x}')

# print(df_new.shape)





In [4]:
mapping_new={
    'no':0,
    'yes':1,
    'male':1,
    'female':0,
    'normal':0,
    'diarrhea':1,
    'constipation':1,
    '+':1,
    '++':2,
    '+++':3,
    'local':1,
    'generalized':2,
    'appendicitis':1,
    'no appendicitis':0,
    'noAppendicitis':0,
    'uncomplicated':0,
    'complicated':1,
}

for col in df_new.columns:
    df_new[col]=df_new[col].replace(mapping_new)
    # print(f'{col} : {df_new[col].isna().sum()}')

# df_new.to_excel('draft2.xlsx',index=False)











C:\Users\ajung\AppData\Local\Temp\ipykernel_19844\4118727067.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_new[col]=df_new[col].replace(mapping_new)


In [5]:
#Chi-Square Test for categorical variables

df_new_cat=df_new[feat_cat]

target=['Diagnosis', 'Severity']
results_cat=[]
for col in df_new_cat.columns:
    if col in target:
        continue
    data = df_new_cat[[col, 'Diagnosis']].dropna()
    x=data.shape[0]
    contingency_table = pd.crosstab(data[col], data['Diagnosis'])  # Create a contingency table
    chi2, p, dof, expected = chi2_contingency(contingency_table)  # Perform Chi-Square test

    n = contingency_table.values.sum()  # Total number of observations
    k = min(contingency_table.shape)  # Smaller of the number of rows or columns
    cramers_v = np.sqrt(chi2 / (n * (k - 1)))


    results_cat.append({
        'Features': col,
        'Chi-Square': chi2,
        'p-value': p,
        'Degrees of Freedom': dof,
        'cramers v': cramers_v,
        'Samples':x

    })

results_df = pd.DataFrame(results_cat)
results_df_sorted = results_df.sort_values(by='cramers v', ascending=False)

results_df_sorted.to_excel('cat_diag.xlsx',index=False)

print(results_df_sorted)



                            Features  Chi-Square       p-value  \
13                       Peritonitis  108.238182  3.136010e-24   
7                       Neutrophilia   67.169617  2.491240e-16   
11                       Free_Fluids   32.476397  1.206490e-08   
14                  Ketones_in_Urine   24.339202  2.122058e-05   
10  Contralateral_Rebound_Tenderness   25.081588  5.495509e-07   
3                             Nausea   23.194358  1.464263e-06   
4                   Loss_of_Appetite   19.235202  1.155627e-05   
9     Ipsilateral_Rebound_Tenderness    8.639745  3.289075e-03   
0                                Sex   10.777933  1.027173e-03   
8               Lower_Right_Abd_Pain    8.182539  4.229553e-03   
1                     Migratory_Pain    6.696014  9.662869e-03   
2                      Coughing_Pain    6.300871  1.206787e-02   
12                        Psoas_Sign    4.001254  4.546643e-02   
6                              Stool    3.042507  2.184379e-01   
15        

In [6]:
results_cont=[]

df_new_cont=df_new[feat_cont]

# print(df_new_cont.columns)
for col in df_new_cont.columns:
    if col in target:
        continue
    data = df_new_cont[[col, 'Diagnosis']].dropna()
    x=data.shape[0]
    group_0=data[data['Diagnosis']==0][col]
    group_1=data[data['Diagnosis']==1][col]
    stat, p_value = mannwhitneyu(group_0, group_1, alternative='two-sided')

    n1, n0 = len(group_1), len(group_0)
    r_rb = 1 - (2 * stat) / (n1 * n0)

    median_0 = np.median(group_0)
    median_1 = np.median(group_1)

    mean_0 = np.mean(group_0)
    mean_1 = np.mean(group_1)
    
    results_cont.append({'Feature': col, 'U_statistic': stat, 'p_value': p_value,'rank-biserial correlation': r_rb,'Mean (Positive Class)':mean_1,
                    'Mean(Negative Class)':mean_0,'Median(Positive Class)':median_1,'Median(Negative Class)':median_0,'Samples':x})

  

results_df = pd.DataFrame(results_cont)
results_df_sorted = results_df.sort_values(by='p_value', ascending=True)


results_df_sorted.to_excel('con_diag.xlsx',index=False)